<h1>Content<span class="tocSkip"></span></h1>

We are working in internet shop, which is selling video games all around the world. We have data about games sales, critic and user scores, genres and platforms from open sources. We need to define success patterns for games.It can help us to bet on potentially popular product and to plan ad campaign.

## The review of the data

In [1]:
#let's define libraries
import pandas as pd
games = pd.read_csv('https://code.s3.yandex.net/datasets/games.csv')
import math
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats as st


In [2]:

display(games.head(10))
games.info()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


### Summary:

In dataset <b>games</b> there are folowing fields:
<ul>
<li> Name — name of a game</li>
<li>Platform — a platform, on which a game was released</li>
<li>Year_of_Release — year of a game realease</li>
<li>Genre — genre of a game</li>  
<li>NA_sales — sales in North America (millions of sold copies)</li>
<li>EU_sales — sales in Europe (millions of sold copies)</li>
<li>JP_sales — sales in Japan (millions of sold copies)</li>
<li>Other_sales — sales in other countries (millions of sold copies)</li>
<li>Critic_Score — scores by critics (maximum 100)</li>
<li>User_Score — scores by users (maximum 10)</li>
<li>Rating — rating by ESRB organization (Entertainment Software Rating Board). This organization defines a rating of computer games and categorizes them by age.</li></ul>


There are 11 columns in dataset, 16715 entries. <ul>
<li>Column Name has an object format, there are 2 missing values. </li>
<li>Column Platform has an object format, there aren't missing values.</li>
<li>Column Year_of_Release has float64 format, there are 269 missing values in it.</li> 
    <li>Column Genre has an object format, there are 2 missing values.</li> 
    <li>Column NA_sales  имеет формат float64, there aren't missing values.</li>
    <li>Column EU_sales имеет формат float64, there aren't missing values.</li>
    <li>Column JP_sales имеет формат float64, there aren't missing values. </li>
    <li>Column Other_sales имеет формат float64, there aren't missing values. </li>
    <li>Column Critic_Score имеет формат float64, there are 8578 missing values. </li>
    <li>Column User_Score has an object format, there are 6701 missing values.
    <li>Column Rating has an object format, there are 6773 missing values. </li></ul>
    
When studying the initial information, the following problews were found in a dataset: Name of columns are not unified and begin with capital letters, there are missing values in columns Name, Year_of_Release, Genre, Critic_Score, User_Score, Raiting. Besides that, a column with numeric values User_Score  has an object format.
 
    

## Preparing to data processing


To process the data in dataset correctly, we need to do:
<ul>
<li>To change names in columns</li>
<li>To convert data to correct types </li>
<li>To process missing values, if it's necessary.</li>
<li>To describe the reasons of missing values</li>
<li>To calculate total sales in all regions and record them in a new column.</li>

### Changing names in columns


In [3]:
games.columns = map(str.lower, games.columns)
display(games.head(5))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


### Converting data to correct types

In [4]:
games['critic_score'] = pd.to_numeric(games['critic_score'], errors='coerce')


When converting the user_score column from object to float8, there is a problem with the column, which in addition to numerical values, contains the tbd value.
There was also a problem when converting the year_of_release column to int32, because there are blank spaces.



### Processing missing values


In [5]:
display(games[games['name'].isnull()])

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


It seems, that the games with null values hadn't official name, because it is 1993 year of release. We should delete these two entries from dataset, because there are no name of games and genre, and a year of release is 1993 - this data is not interesting for us.

In [6]:
games = games.drop(games[games['name'].isnull()].index)
games = games.reset_index(drop=True)
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16713 entries, 0 to 16712
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Let's fill null values, replace missing values with 0 and convert the data to int in the year_of_release column.

In [7]:
games['year_of_release']=games['year_of_release'].fillna(0)
games.info()
display(games[games['year_of_release']==0])

#from the sample we can see that some games contain the year of release in the title. 
#Let's extract the year of release using a mask of 4 digits to a new column.

games['year'] = games.loc[(games['year_of_release']==0), 'name'].str.extract("(\d\d\d\d)", expand=True)
display(games[games['year_of_release']==0])

games['year']=games['year'].fillna(0)
games.loc[(games['year_of_release']==0), 'year_of_release'] = games['year']

games.info()
#let's delete unnecessary column
games = games.drop(columns='year')
display(games[games['year_of_release']==0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16713 entries, 0 to 16712
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
183,Madden NFL 2004,PS2,0.0,Sports,4.26,0.26,0.01,0.71,94.0,8.5,E
377,FIFA Soccer 2004,PS2,0.0,Sports,0.59,2.36,0.04,0.51,84.0,6.4,E
456,LEGO Batman: The Videogame,Wii,0.0,Action,1.80,0.97,0.00,0.29,74.0,7.9,E10+
475,wwe Smackdown vs. Raw 2006,PS2,0.0,Fighting,1.57,1.02,0.00,0.41,NaN,NaN,NaN
609,Space Invaders,2600,0.0,Shooter,2.36,0.14,0.00,0.03,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16371,PDC World Championship Darts 2008,PSP,0.0,Sports,0.01,0.00,0.00,0.00,43.0,tbd,E10+
16403,Freaky Flyers,GC,0.0,Racing,0.01,0.00,0.00,0.00,69.0,6.5,T
16446,Inversion,PC,0.0,Shooter,0.01,0.00,0.00,0.00,59.0,6.7,M
16456,Hakuouki: Shinsengumi Kitan,PS3,0.0,Adventure,0.01,0.00,0.00,0.00,NaN,NaN,NaN


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,year
183,Madden NFL 2004,PS2,0.0,Sports,4.26,0.26,0.01,0.71,94.0,8.5,E,2004
377,FIFA Soccer 2004,PS2,0.0,Sports,0.59,2.36,0.04,0.51,84.0,6.4,E,2004
456,LEGO Batman: The Videogame,Wii,0.0,Action,1.80,0.97,0.00,0.29,74.0,7.9,E10+,NaN
475,wwe Smackdown vs. Raw 2006,PS2,0.0,Fighting,1.57,1.02,0.00,0.41,NaN,NaN,NaN,2006
609,Space Invaders,2600,0.0,Shooter,2.36,0.14,0.00,0.03,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16371,PDC World Championship Darts 2008,PSP,0.0,Sports,0.01,0.00,0.00,0.00,43.0,tbd,E10+,2008
16403,Freaky Flyers,GC,0.0,Racing,0.01,0.00,0.00,0.00,69.0,6.5,T,NaN
16446,Inversion,PC,0.0,Shooter,0.01,0.00,0.00,0.00,59.0,6.7,M,NaN
16456,Hakuouki: Shinsengumi Kitan,PS3,0.0,Adventure,0.01,0.00,0.00,0.00,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16713 entries, 0 to 16712
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  object 
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
 11  year             16713 non-null  object 
dtypes: float64(5), object(7)
memory usage: 1.5+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
456,LEGO Batman: The Videogame,Wii,0,Action,1.80,0.97,0.00,0.29,74.0,7.9,E10+
609,Space Invaders,2600,0,Shooter,2.36,0.14,0.00,0.03,NaN,NaN,NaN
627,Rock Band,X360,0,Misc,1.93,0.33,0.00,0.21,92.0,8.2,T
657,Frogger's Adventures: Temple of the Frog,GBA,0,Adventure,2.15,0.18,0.00,0.07,73.0,tbd,E
677,LEGO Indiana Jones: The Original Adventures,Wii,0,Action,1.51,0.61,0.00,0.21,78.0,6.6,E10+
...,...,...,...,...,...,...,...,...,...,...,...
16346,Agarest Senki: Re-appearance,PS3,0,Role-Playing,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16403,Freaky Flyers,GC,0,Racing,0.01,0.00,0.00,0.00,69.0,6.5,T
16446,Inversion,PC,0,Shooter,0.01,0.00,0.00,0.00,59.0,6.7,M
16456,Hakuouki: Shinsengumi Kitan,PS3,0,Adventure,0.01,0.00,0.00,0.00,NaN,NaN,NaN


In [8]:
games['year_of_release'] = games['year_of_release'].astype('int32')
games.info()
display(games.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16713 entries, 0 to 16712
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  int32  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(5), int32(1), object(5)
memory usage: 1.3+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


In [9]:
display(games[games['critic_score'].isnull()])
display(games[(games['critic_score'].isnull())&(games['year_of_release']==2016)])

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN
10,Nintendogs,DS,2005,Simulation,9.05,10.95,1.93,2.74,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16708,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16709,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16710,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16711,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
108,Pokemon Sun/Moon,3DS,2016,Role-Playing,2.98,1.45,2.26,0.45,NaN,NaN,NaN
1215,Ratchet & Clank (2016),PS4,2016,Platform,0.50,0.76,0.04,0.24,NaN,NaN,NaN
1274,Doom (2016),PS4,2016,Shooter,0.54,0.69,0.02,0.24,NaN,NaN,NaN
1465,Yokai Watch 3,3DS,2016,Action,0.00,0.00,1.33,0.00,NaN,NaN,NaN
1474,The Elder Scrolls V: Skyrim,PS4,2016,Role-Playing,0.48,0.59,0.04,0.21,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16697,The Longest 5 Minutes,PSV,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16701,Strawberry Nauts,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16708,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16710,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN


As we can see, in rows, which contain null data in column 'critic_score', there is no data in 'user_score' and 'raiting'. Perhaps, in this dataframe there was no data about it initially. Also, 270 rows contain data with games of 2016 year of release, so maybe this data was received partly before publishing.

it's necessaty to decide,  how we can fill missing values in columns 'critic_score', 'user_score' and 'raiting'. We think, that the rating can be left as it is, and for plotting it is better to convert column 'user_score' from object to float. 
To do this, it is necessary to solve the issue of the tbd value.
Let's consider th unique values of 'critic_score' and 'user_score'.

In [10]:
display(games['critic_score'].unique())
display(games['user_score'].unique())

array([76., nan, 82., 80., 89., 58., 87., 91., 61., 97., 95., 77., 88.,
       83., 94., 93., 85., 86., 98., 96., 90., 84., 73., 74., 78., 92.,
       71., 72., 68., 62., 49., 67., 81., 66., 56., 79., 70., 59., 64.,
       75., 60., 63., 69., 50., 25., 42., 44., 55., 48., 57., 29., 47.,
       65., 54., 20., 53., 37., 38., 33., 52., 30., 32., 43., 45., 51.,
       40., 46., 39., 34., 35., 41., 36., 28., 31., 27., 26., 19., 23.,
       24., 21., 17., 22., 13.])

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

The abbreviation 'tbd' there is in the column with user_score only. Tbd means "to be determined", the user rating has not yet been determined. So, it does not exist yet. Based on this, we can easily replace omissions and tbd with zeros, and when plotting graphs, only data greater than zero should be taken.


In [11]:
games['critic_score']=games['critic_score'].fillna(0)


games.loc[(games['user_score']=='tbd'), 'user_score'] = 0

games['user_score'] = games['user_score'].astype('float64')

games['user_score'] = games['user_score'].fillna(0)
games.info()
display(games.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16713 entries, 0 to 16712
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  int32  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     16713 non-null  float64
 9   user_score       16713 non-null  float64
 10  rating           9949 non-null   object 
dtypes: float64(6), int32(1), object(4)
memory usage: 1.3+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,0.0,0.0,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,0.0,0.0,NaN
5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,0.0,0.0,NaN
6,New Super Mario Bros.,DS,2006,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,0.0,0.0,NaN


### Calculating total sales

In [13]:
games['total_sales'] = games[['na_sales','eu_sales','jp_sales','other_sales']].sum(axis=1)
display(games.head(20))
games['total_sales'].unique()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E,82.54
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,0.0,0.0,NaN,40.24
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,0.0,0.0,NaN,31.38
5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,0.0,0.0,NaN,30.26
6,New Super Mario Bros.,DS,2006,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E,29.80
7,Wii Play,Wii,2006,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E,28.91
8,New Super Mario Bros. Wii,Wii,2009,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E,28.32
9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,0.0,0.0,NaN,28.31


array([8.254e+01, 4.024e+01, 3.552e+01, ..., 2.000e-02, 1.000e-02,
       0.000e+00])

### Summary
Мы изучили пропуски и сделали предположения о причинах их появления в датасете. Был изменен формат данных, а также убрана пометка tbd из столбца user_score. Добавлена новая колонка total_sales с итоговой суммой. Оказалось, что весь датасет отсортирован в порядке убывания суммы общих продаж.


## Research data analysis
Посмотрим, сколько игр выпускалось в разные годы. 
Определим, как менялись продажи по платформам. Выберем платформы с наибольшими суммарными продажами и постройте распределение по годам.        
Возьмем данные за соответствующий актуальный период и определим, какие платформы лидируют по продажам, растут или падают. Выберем несколько потенциально прибыльных платформ.

Построим график «ящик с усами» по глобальным продажам игр в разбивке по платформам.
Посмотрим, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Построим диаграмму рассеяния и посчитаем корреляцию между отзывами и продажами. 


Посмотрим на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?


### Анализ игр по годам выпуска

In [ ]:
games_count = games[games['year_of_release']>0].pivot_table(values = ['name'], index = ['year_of_release'],\
                                                            aggfunc = 'count').reset_index()
display(games_count)


### Анализ продаж игр по платформам

Определим платформы с наибольшими суммарными продажами.
Для этого сделаем сводную таблицу pivot table с платформами


In [ ]:
platform_rating = games.pivot_table(values = ['total_sales'], index = ['platform'],aggfunc = 'sum')
display(platform_rating)
platform_rating = platform_rating.reset_index()\
      .sort_values(['total_sales'], ascending= 0)
display(platform_rating)

platform_rating.columns = ['platform','total_sales']



In [ ]:
#Выберем список наиболее популярных платформ
top_platform = platform_rating[(platform_rating['total_sales']>250)]
display(top_platform)
names_top = top_platform['platform'].tolist()

#Выберем на основе данного списка данные из общего датасета
filtered_platform_data = games[(games['platform'].isin(names_top))]
display(filtered_platform_data.head(5))




In [ ]:
#Построим pivot_table с распределением популярных платформ по годам
platform_rating_top = filtered_platform_data.pivot_table(values = ['total_sales'], columns = ['platform'], index = ['year_of_release'],aggfunc = 'sum')
platform_rating_top.columns = platform_rating_top.columns.droplevel(0)
platform_rating_top = platform_rating_top.rename_axis(None, axis=1)
display(platform_rating_top)

cmap = cm.get_cmap('Spectral')

platform_rating_top.plot(kind='bar', stacked=True, width=0.85)
plt.title("Распределение продаж популярных платформ по годам")
plt.xlabel("Года")
plt.ylabel("Количество проданных копий, млн")



plt.legend(bbox_to_anchor=(1, 1))
plt.show()

Мы выбрали самые популярные платформы и сравнили продажи по годам. Из графиков мы видим, что средний срок "жизни" платформы 8-10 лет. Из этого следует, что если мы возьмем выборку по половине срока "жизни" платформы, 5 лет, из этой выборки будет точно понятно, находится ли платформа на пике популярности, набирает или перестает быть популярной. 

Сделааем выборку по всем платформам за предыдущие 5 лет

In [ ]:
filtered_platform_year = games[(games['year_of_release']>2011)]
platform_actual = filtered_platform_year.pivot_table(values = ['total_sales'], columns = ['platform'], index = ['year_of_release'],aggfunc = 'sum')

platform_actual.columns = platform_actual.columns.droplevel(0)
platform_actual = platform_actual.rename_axis(None, axis=1)

display(platform_actual)
display(platform_actual.columns)
platform_actual.plot(kind='bar', stacked=True, width=0.85)
plt.title("Распределение продаж платформ 2012-2016гг")
plt.xlabel("Года")
plt.ylabel("Количество проданных копий, млн")
plt.legend(bbox_to_anchor=(1, 1))
plt.show()



2016 год выдался довольно непростым для игровых платформ - продажи упали в целом. Судя по графику, можно выделить платформы находящиеся к 2017 году на пике - XOne и PS4. Остальные платформы из представленных уходят с рынка.

In [ ]:

plt.rcParams['figure.figsize'] = (15,10)
sns.boxplot(data=platform_actual)
plt.title("Распределение объемов продаж платформ 2012-2016гг")
plt.xlabel("Платформы")
plt.ylabel("Количество проданных копий, млн")


В построенных графиках мы видим распределение данных по продажам платформ за выбранный отрезок времени. Графики очень сильно различаются, в данных графиков WiiU, PC, Wii присутствуют выбросные данные, причем у платформы WiiU минимальные, а у платформ PS и Wii максимальные. Данные Wii и WiiU находятся по разным стороным графиков.Возможно, эта неровность вызвана тем, что WiiU - это следующее поколение платформы Wii и спад популярности одной вызван ростом популярности другой платформы. Платформы на пике популярности PS4 и XOne не имеют выбросных данных, 1 и 3 квартили примерно симметричны.

### Оценка зависимости продаж от отзывов 
Изучим влияние критиков на продажи внутри популярной платформы PS4. Построим диаграмму рассеяния и изучим корелляцию

In [ ]:
data_ps4 = games[(games['platform'] =='PS4')&(games['year_of_release'] > 2011)&(games['critic_score']>0)&(games['user_score']>0)]
display(data_ps4.head(5))

#scatterplot = sns.scatterplot(data=data_ps4, x='total_sales', y=['critic_score', 'user_score'])
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = data_ps4['total_sales']
y = data_ps4['critic_score']
z = data_ps4['user_score']

ax.scatter(x, y, z)
ax.set_xlabel("Проданные копии, млн")
ax.set_ylabel("Оценки критиков")
ax.set_zlabel("Оценки пользователей")

plt.show()

fig, ax = plt.subplots(2, figsize=(10, 6))
ax[0].scatter(x = data_ps4['total_sales'], y = data_ps4['critic_score'])
ax[0].set_xlabel("Проданные копии, млн")

ax[0].set_ylabel("Оценки критиков")

ax[1].scatter(x = data_ps4['total_sales'], y = data_ps4['user_score'])
ax[1].set_xlabel("Проданные копии, млн")
ax[1].set_ylabel("Оценки пользователей")

plt.show()


corr = data_ps4[['total_sales', 'critic_score', 'user_score']].corr()
print('Коэффициент корелляции между продажами игр на платформе PS4 и оценками критиков и пользователей:' )
display(corr)

Мы видим примерно одинаковую картину зависимости продаж от оценок критиков и пользователей. В диаграмме рассеяния мы наблюдаем прямую корреляцию продажами между оценками критиков и незначительную обратную между оценками пользователей и продажами. Полагаем, в остальных платформах имеется та же картина - их продажи не зависят от оценок пользователей, но немного зависят от оценок критиков

In [ ]:
#проверим эту закономерность на всех платформах
common_data = games[(games['year_of_release'] > 2011)&(games['critic_score']>0)&(games['user_score']>0)]
fig, ax = plt.subplots(2, figsize=(10, 6))
ax[0].scatter(x = common_data['total_sales'], y = common_data['critic_score'])
ax[0].set_xlabel("Проданные копии, млн")

ax[0].set_ylabel("Оценки критиков")

ax[1].scatter(x = common_data['total_sales'], y = common_data['user_score'])
ax[1].set_xlabel("Проданные копии, млн")
ax[1].set_ylabel("Оценки пользователей")

In [ ]:
corr = common_data[['total_sales', 'critic_score', 'user_score']].corr()
print('Коэффициент корелляции между продажами всех игр и оценками критиков и пользователей:' )
display(corr)

Наши предположения верны. Графики рассеяния PS4  и всех игр схожи. Корелляция всех игр с оценками критиков даже еще меньше - 30%

### Анализ игр по жанрам

In [ ]:
actual_genres = filtered_platform_year.pivot_table(values = ['total_sales'], index = ['genre'],aggfunc = ['count', 'sum']).reset_index()
actual_genres.columns = actual_genres.columns.droplevel(1)
actual_genres = actual_genres.rename_axis(None, axis=1)
display(actual_genres)
actual_genres['sales_to_release'] = actual_genres['sum']/actual_genres['count']
actual_genres.plot(x = 'genre', y = 'sales_to_release', kind='bar', width=0.85)
plt.title("Распределение продаж игр по жанрам в актуальном отрезке времени")
plt.xlabel("Жанры")
plt.ylabel("Количество проданных копий, млн")
plt.legend(bbox_to_anchor=(1, 1)) 
plt.show()


Наиболее популярный жанр - Shooter, за ним Platform и Sports. Самые непопулярные - Puzzle и Adventure

## Составим портрет пользователя каждого региона

Определите для пользователя каждого региона (NA, EU, JP):
Самые популярные платформы (топ-5). Опишите различия в долях продаж.
Самые популярные жанры (топ-5). Поясните разницу.
Влияет ли рейтинг ESRB на продажи в отдельном регионе?

In [ ]:
actual_platforms_region = filtered_platform_year.pivot_table(values = ['na_sales', 'eu_sales', 'jp_sales'], index = ['platform'],aggfunc = 'sum')
display(actual_platforms_region)
actual_platforms_region.plot(kind='bar', width=0.85)
plt.title("Распределение продаж платформ  по регионам в актуальном отрезке времени")
plt.xlabel("Платформы")
plt.ylabel("Количество проданных копий, млн")


В целом, популярные платформы в Европе и Северной Америке не отличаются друг от друга. Разница только в объемах продаж. Самые высокие продажи в Северной Америке и Европе. Самые низкие объемы продаж различных платформ у Японии. И здесь главное отличие в популярных платформах, так азиатский рынок довольно своеобразен и на него не действуют законы продаж, принятые на Западе. Полагаем, это также будет видно по популярным жанрам игр.

Топ-5 платформ в Северной Америке:
<ol>
<li> X360</li>
<li> PS4</li>    
<li> PS3</li>    
<li> XOne</li> 
<li> 3DS</li> </ol>   

Топ-5 платформ в Европе:
<ol>
<li> X360</li>
<li> PS4</li>    
<li> PS3</li>    
<li> 3DS</li> 
<li> XOne</li> </ol>  

Топ-5 платформ в Японии:
<ol>
<li> 3DS</li>
<li> PS3</li>    
<li> PSV</li>    
<li> PS4</li> 
<li> WiiU</li> </ol>  

In [ ]:
actual_genres_region = filtered_platform_year.pivot_table(values = ['na_sales', 'eu_sales', 'jp_sales'], index = ['genre'],aggfunc = 'sum')
display(actual_genres_region)
actual_genres_region.plot(kind='bar', width=0.85)
plt.title("Распределение продаж игр по жанрам по регионам в актуальном отрезке времени")
plt.xlabel("Жанры игр")
plt.ylabel("Количество проданных копий, млн")

В этом графике мы видим подтверждение, что японский рынок игр отличается от западного. В части некоторых жанров регионы схожи, но есть и глобальные отличия.
Топ-5 игровых жанров в Северной Америке:
<ol>
<li> Action	</li>
<li> Shooter</li>    
<li> Sports</li>    
<li> Role-playing</li> 
<li> Misc</li> </ol>   

Топ-5 игровых жанров в Европе:
<ol>
<li> Action	</li>
<li> Shooter</li>    
<li> Sports</li>    
<li> Role-playing</li> 
<li> Racing</li> </ol>    

Топ-5 платформ в Японии:
<ol>
<li> Role-playing</li>
<li> Action	</li>
<li> Misc</li>    
<li> Simulation</li>    
<li> Fighting</li> </ol>  


Соберем сводную таблицу по количеству продаж игр по рейтингам

In [ ]:


games_rating = filtered_platform_year[~filtered_platform_year['rating'].isnull()]
games_not_rated = filtered_platform_year[filtered_platform_year['rating'].isnull()]
actual_rating_region = games_rating.pivot_table(values = ['na_sales', 'eu_sales', 'jp_sales'],\
                                                          index = ['rating'],aggfunc = 'sum')

actual_rating_region.columns = ['sum_na', 'sum_eu', 'sum_jp']

display(actual_rating_region)

Так как игры в каких-то регионах не продавались, количество их релизов было разным. Почитаем количество их релизов по каждому региону и смержим в общую таблицу.

In [ ]:

na_rating = games_rating[games_rating['na_sales']!=0]
na_rating_count = na_rating.pivot_table(values = 'na_sales',\
                                                          index = ['rating'],aggfunc = 'count')
na_rating_count.columns = ['na_count']
actual_rating_region = actual_rating_region.merge(na_rating_count, on=['rating'],how='outer')

eu_rating = games_rating[games_rating['eu_sales']!=0]
eu_rating_count = eu_rating.pivot_table(values = 'eu_sales',\
                                                          index = ['rating'],aggfunc = 'count')
eu_rating_count.columns = ['eu_count']
actual_rating_region = actual_rating_region.merge(eu_rating_count, on=['rating'],how='left')

jp_rating = games_rating[games_rating['jp_sales']!=0]
jp_rating_count = jp_rating.pivot_table(values = 'jp_sales',\
                                                          index = ['rating'],aggfunc = 'count')
jp_rating_count.columns = ['jp_count']
actual_rating_region = actual_rating_region.merge(jp_rating_count, on=['rating'],how='outer')



Теперь посчитаем количество продаж на каждый релиз по регионам и построим график

In [ ]:
#посчитаем сумму продаж на каждый релиз по регионам
actual_rating_region['sales_per_release_na'] = actual_rating_region['sum_na']/actual_rating_region['na_count']
actual_rating_region['sales_per_release_eu'] = actual_rating_region['sum_eu']/actual_rating_region['eu_count']
actual_rating_region['sales_per_release_jp'] = actual_rating_region['sum_jp']/actual_rating_region['jp_count']
display(actual_rating_region)

actual_rating_region = actual_rating_region.drop(columns=['sum_na', 'sum_eu', 'sum_jp', 'na_count', 'eu_count', 'jp_count'])

actual_rating_region.plot(kind='bar', width=0.85)
plt.title("Распределение продаж игр по рейтингу по регионам в актуальном отрезке времени")
plt.xlabel("Рейтинг")
plt.ylabel("Количество проданных копий на один релиз, млн")

Для начала разберёмся, что означают рейтинги ESRB:

<ul><li>E - everyone</li>
<li>E10+ - everyone 10+</li>
<li>T - teen (13+)</li>
<li>M - mature (17+)</li>
<li>A - adults (18+)</li>
<li>RP - rating pending (рейтинг пока не присвоен)</li></ul>

Согласно нашему исследованию, Япония бьет рекорды по количеству проданных копий любого рейтинга. E, E10+, M наиболее популярны. А игры для подростков (T)продаются хуже. 
В Европе игры для всех и игры для подростков (T) продаются чуть лучше, чем  с возрастным цензом 17 и 10 лет (M и E10+).
В Северной Америке игры для всех (E) и игры c возрастным цензом (M) продаются лучше остальных.

Теперь проведем исследование для игр без рейтинга

In [ ]:
#создадим пустой датафрейм и дадим названия столбцов
games_not_rated_region = pd.DataFrame(columns=['sales_per_release_na',\
                                               'sales_per_release_eu', 'sales_per_release_jp'])

#посчитаем сумму продаж и количество релизов для каждого региона
not_rated_na_sum = games_not_rated.loc[(games_not_rated['na_sales']!=0), 'na_sales'].sum()
not_rated_na_count = games_not_rated.loc[(games_not_rated['na_sales']!=0), 'na_sales'].count()
games_not_rated_na = not_rated_na_sum/not_rated_na_count
print(not_rated_na_sum)
print(not_rated_na_count)

not_rated_eu_sum = games_not_rated.loc[(games_not_rated['eu_sales']!=0), 'eu_sales'].sum()
not_rated_eu_count = games_not_rated.loc[(games_not_rated['eu_sales']!=0), 'eu_sales'].count()
print(not_rated_eu_sum)
print(not_rated_eu_count)

games_not_rated_eu = not_rated_eu_sum/not_rated_eu_count

not_rated_jp_sum = games_not_rated.loc[(games_not_rated['jp_sales']!=0), 'jp_sales'].sum()
not_rated_jp_count = games_not_rated.loc[(games_not_rated['jp_sales']!=0), 'jp_sales'].count()
print(not_rated_jp_sum)
print(not_rated_jp_count)

games_not_rated_jp = not_rated_jp_sum/not_rated_jp_count


#заполним датасет
games_not_rated_region = games_not_rated_region.append({'sales_per_release_na': \
                                                        games_not_rated_na, 'sales_per_release_eu':\
                                                        games_not_rated_eu, 'sales_per_release_jp':\
                                                        games_not_rated_jp}, ignore_index=True)


display(games_not_rated_region)
games_not_rated_region.plot(kind='bar', width=0.85)
plt.title("Распределение продаж игр без рейтинга по регионам в актуальном отрезке времени")
plt.xlabel("Регионы продажи игр: Северная Америка, Европа, Япония")
plt.ylabel("Количество проданных копий на один релиз, млн")

Сравнив графики, мы видим, что во всех странах игры без рейтинга обладают такой же популярностью, как и игры с рейтингом, во всех регионах, кроме Японии.

## Hypothesis checking
<b>Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.</b>

Сформулируем нулевую гипотезу:Средние пользовательские рейтинги платформ Xbox One и PC в представленной выборке одинаковые.

Сформулируем альтернативную гипотезу: Средние пользовательские рейтинги платформ Xbox One и PC в представленной выборке различаются.

In [ ]:
data_actual_xone = filtered_platform_year[(filtered_platform_year['platform'] == 'XOne')&(~filtered_platform_year['user_score'].isnull())]
data_actual_pc = filtered_platform_year[(filtered_platform_year['platform'] == 'PC')&(~filtered_platform_year['user_score'].isnull())]
print('cреднее XOne: ', data_actual_xone['user_score'].mean())
print('cреднее PC: ', data_actual_pc['user_score'].mean())

alpha = 0.05

results = st.ttest_ind(data_actual_xone['user_score'],data_actual_pc['user_score'])

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

<b>Средние пользовательские рейтинги жанров Action и Sports разные.</b>

Сформулируем нулевую гипотезу:Средние пользовательские рейтинги жанров Action и Sports  в представленной выборке одинаковые.

Сформулируем альтернативную гипотезу: Средние пользовательские рейтинги жанров Action и Sports  в представленной выборке различаются.


In [ ]:
data_actual_action = filtered_platform_year[(filtered_platform_year['genre'] == 'Action')&(~filtered_platform_year['user_score'].isnull())]
data_actual_sports = filtered_platform_year[(filtered_platform_year['genre'] == 'Sports')&(~filtered_platform_year['user_score'].isnull())]
print('cреднее Action: ', data_actual_action['user_score'].mean())
print('cреднее Sports: ', data_actual_sports['user_score'].mean())

alpha = 0.05

results = st.ttest_ind(data_actual_action['user_score'],data_actual_sports['user_score'])

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

### Вывод
При проверке гипотезы о том, что средние пользовательские рейтинги платформ Xbox One и PC одинаковые, для сравнения была выбрана гипотеза о равенстве средних двух генеральных совокупностей, потому что для исследования берутся две неравные группы с пользовательскими оценками игр разных платформ, обладающих разными признаками.

Исследование показало, что с вероятностью 95% средние пользовательские рейтинги платформ Xbox One и PC одинаковые (у нас не получилось отвергнуть нулевую гипотезу)

При проверке гипотезы о том, что средние пользовательские рейтинги жанров Action и Sports разные, была выбрана гипотеза о равенстве средних двух генеральных совокупностей. 

Исследование показало, что средние пользовательские рейтинги жанров Action и Sports разные (мы отвергли нулевую гипотезу)

## Общий вывод

В результате нашего исследования мы выявили определяющие успешность игры закономерности:
<ol>
    <li>Платформа игры. 
    На конец 2016 года набирают популярность платформы PS4 и XBox One. Это утверждение имеет значение для общего рынка игр и Северной Америки и Европы в частности. В Японии не все так однозначно и большинство предпочитает другие, не представленные в выборке платформы </li>
    <li>Жанр игры.
    Наиболее популярный жанр - Simulation, за ним Platform и Sports. Самые непопулярные - Puzzle и Adventure</li>
    <li>Регион продажи игры.
    Исследование показало, что рынки игр Северной Америки и Европы мало отличаются, а в Японии рынок игр, хоть и имеет какие-то общие закономерности, в целом отличается от западных</li>
    <li>Рейтинг
    Мы выяснили, что в Японии игры с рейтингом продаются лучше, чем игры без рейтинга, в остальных же регионах рейтинг не влияет на успешность.</li>
</ol>